In [1]:
# %%
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from pycaret.regression import *

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
model_dict = dict()

In [3]:
# %%
train = pd.read_csv('./data/train.csv')
test =  pd.read_csv('./data/test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [4]:
item_dict = {
    '감귤' : 'TG',
    '꽃양배추와 브로콜리(broccoli)' : 'BC',
    '무' : 'RD',
    '당근' : 'CR',
    '양배추' : 'CB'
}

item_list = ['감귤', '꽃양배추와 브로콜리(broccoli)', '무', '당근', '양배추']
trade['item'] = [item_dict[e] if e in item_list else 'None' for e in trade['품목명']]

In [5]:
# %%
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

trade['year'] = trade['기간'].apply(lambda x : int(x[0:4]))
trade['month'] = trade['기간'].apply(lambda x : int(x[5:7]))


result = train.groupby(['year', 'month', 'item'])['supply(kg)'].sum().reset_index()


In [6]:
r_trade = trade.merge(result, on=['year', 'month','item'], how='outer')
r_trade['전체 거래량'] = r_trade['수출 중량'] + r_trade['수입 중량']

trade_use_list = ['TG', 'BC']
qual_col = ['item', 'corporation', 'location','day_name']


In [10]:
train_x

,week,year,month,day,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,supply(kg)_y,...,location_J,location_S,day_name_Friday,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,price(원/kg)
0,1,2019,1,1,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,0,0,0,1,0,0.0
1,1,2019,1,2,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,0,0,0,0,1,0.0
2,1,2019,1,3,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,0,0,1,0,0,1728.0
3,1,2019,1,4,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,1,0,0,0,0,0,0,1408.0
4,1,2019,1,5,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,1,0,0,0,0,1250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15225,9,2023,3,2,0.0,0.0,0.0,0.0,0.0,495802.3,...,1,0,0,0,0,0,1,0,0,1904.0
15226,9,2023,3,3,0.0,0.0,0.0,0.0,0.0,495802.3,...,1,0,1,0,0,0,0,0,0,1622.0
15227,9,2023,3,1,0.0,0.0,0.0,0.0,0.0,495802.3,...,0,1,0,0,0,0,0,0,1,4235.0
15228,9,2023,3,2,0.0,0.0,0.0,0.0,0.0,495802.3,...,0,1,0,0,0,0,1,0,0,3960.0


In [9]:
# TG, BC, RD, CR, CB
item = 'TG'

if item in trade_use_list:
    m_train = train[train['item']==item].merge(r_trade[r_trade['item']==item], on=['year', 'month', 'item'], how='outer')
    m_train = m_train.dropna(subset=['ID'])
    m_train = m_train.fillna(0)

    train_x = m_train.drop(columns=['ID', 'timestamp', 'supply(kg)_x', 'price(원/kg)','Date'])

else: 
    m_train = train[train['item']==item]
    train_x = m_train.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)','Date'])
    if item == 'TG': train_x = m_train['day_name_Sunday', '수출 금액', '무역수지', '전체 거래량', '수출 중량', 'corporation_C']
train_y = m_train['price(원/kg)']

train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
if item in trade_use_list:
    train_x = train_x.drop(columns=['기간','품목명'])
    
train_x = pd.concat([train_x, train_y], axis=1)

s = setup(train_x, target = 'price(원/kg)', 
        session_id = 123, 
        fold = 10, 
        normalize = True, 
        normalize_method = 'minmax', 
        train_size=0.8) ##nomalize에 robust, zscore 가능
# normalise를 robust, zscore로 하면 rmse 너무 높아짐 (1200 수준). minmax 가 680 수준
#fole 수나 train_size를 바꿔보자

# %%
best_model = compare_models()

,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(15230, 27)"
4,Transformed data shape,"(15230, 27)"
5,Transformed train set shape,"(12184, 27)"
6,Transformed test set shape,"(3046, 27)"
7,Numeric features,26
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,841.7148,2317098.4748,1520.0750,0.7031,2.8428,0.1770,0.2990
catboost,CatBoost Regressor,876.3559,2396286.6703,1545.9779,0.6929,3.0649,0.1802,0.4390
rf,Random Forest Regressor,792.4903,2411926.5141,1550.5628,0.6909,2.2379,0.1663,0.4450
xgboost,Extreme Gradient Boosting,890.0473,2548535.7125,1593.1915,0.6734,2.9969,0.1876,0.0430
gbr,Gradient Boosting Regressor,1014.4109,2826664.0387,1679.3410,0.6379,3.1608,0.2016,0.2330
et,Extra Trees Regressor,842.5952,2846133.2523,1685.2516,0.6354,2.1939,0.1838,0.4770
knn,K Neighbors Regressor,1139.8502,3474081.8500,1862.6335,0.5548,2.4724,0.2956,0.0210
ridge,Ridge Regression,1508.5520,4087655.4014,2021.0367,0.4766,3.6136,0.3879,0.0110
br,Bayesian Ridge,1508.1878,4087713.1179,2021.0544,0.4766,3.6137,0.3874,0.0120
lr,Linear Regression,1510.3856,4088892.1632,2021.3410,0.4765,3.6133,0.3905,0.0250


In [72]:
reg  = create_model('rf')
model_dict[f'{item}'] = reg

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,403.5700,549420.8579,741.2293,0.9327,0.1773,0.1494
1,377.9723,490164.4929,700.1175,0.9363,0.1647,0.1345
2,399.2835,553101.3635,743.7078,0.9311,0.1773,0.1478
3,407.3248,513589.0945,716.6513,0.9296,0.1846,0.1545
4,377.0908,488515.3367,698.9387,0.9340,0.1667,0.1350
5,375.4105,482436.6422,694.5766,0.9368,0.2983,0.1357
6,401.3254,555518.6575,745.3312,0.9313,0.1679,0.1424
7,420.0710,744117.0786,862.6222,0.9082,0.1849,0.1480
8,389.4202,523587.1325,723.5932,0.9343,0.1709,0.1401


In [71]:
model_dict

{'TG': LGBMRegressor(n_jobs=-1, random_state=123),
 'BC': LGBMRegressor(n_jobs=-1, random_state=123),
 'RD': RandomForestRegressor(n_jobs=-1, random_state=123),
 'CR': LGBMRegressor(n_jobs=-1, random_state=123),
 'CB': RandomForestRegressor(n_jobs=-1, random_state=123)}

# for Test set

In [56]:
# %%
import datetime
import holidays

# %%
#train = train[(train['supply(kg)'] != 0) & (train['supply(kg)'] != 0)] # 이거차이도 큼!!! 거의 rmse 1200에서 680으로 줄어듦. 근데 이건 validation에만이고 test는 이거하면 정반대로됨 ㅠ...

# %%
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# %%
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [68]:
test_x

,year,month,day,week,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,전체 거래량,item_TG,corporation_A,location_J,day_name_Saturday
0,2023,3,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1


In [64]:
# TG, BC, RD, CR, CB
predictions = []
for i in range(2,test_re.shape[0]):
    sample_test_re = test_re.iloc[i:i+1,:]
    item = sample_test_re['item'][0]
    if item in trade_use_list:
        m_test_re = sample_test_re[sample_test_re['item']==item].merge(r_trade[r_trade['item']==item], on=['year', 'month', 'item'], how='outer')
        m_test_re = m_test_re.dropna(subset=['ID'])
        m_test_re = m_test_re.fillna(0)
        test_x = m_test_re.drop(columns=['ID', 'timestamp', 'supply(kg)','Date'])
    
    else: 
        m_test_re = sample_test_re[sample_test_re['item']==item]
        test_x = m_test_re.drop(columns=['ID', 'timestamp', 'supply(kg)','Date'])
        if item == 'TG': test_x = m_test_re['day_name_Sunday', '수출 금액', '무역수지', '전체 거래량', '수출 중량', 'corporation_C']
    
    test_x = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)
    if item in trade_use_list:
        test_x = test_x.drop(columns=['기간','품목명'])
    
    reg = model_dict[item]
    print(test_x)
    prediction = predict_model(reg, data = test_x)
    predictions.append(prediction)

KeyError: 0

In [8]:
m_test = test_re.merge(trade, on=['year', 'month', 'item'], how='outer')
m_test = m_test.dropna(subset=['ID'])
m_test = m_test.fillna(0)

In [9]:
m_test

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4.0,2023-03-04,9.0,Saturday,0,0,0.0,0.0,0.0,0.0,0.0
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5.0,2023-03-05,9.0,Sunday,0,0,0.0,0.0,0.0,0.0,0.0
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6.0,2023-03-06,10.0,Monday,0,0,0.0,0.0,0.0,0.0,0.0
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7.0,2023-03-07,10.0,Tuesday,0,0,0.0,0.0,0.0,0.0,0.0
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8.0,2023-03-08,10.0,Wednesday,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,BC_E_S_20230327,2023-03-27,BC,E,S,2023,3,27.0,2023-03-27,13.0,Monday,0,0,0.0,0.0,0.0,0.0,0.0
1088,BC_E_S_20230328,2023-03-28,BC,E,S,2023,3,28.0,2023-03-28,13.0,Tuesday,0,0,0.0,0.0,0.0,0.0,0.0
1089,BC_E_S_20230329,2023-03-29,BC,E,S,2023,3,29.0,2023-03-29,13.0,Wednesday,0,0,0.0,0.0,0.0,0.0,0.0
1090,BC_E_S_20230330,2023-03-30,BC,E,S,2023,3,30.0,2023-03-30,13.0,Thursday,0,0,0.0,0.0,0.0,0.0,0.0


In [10]:
# %%
from sklearn.preprocessing import LabelEncoder
train_x = m_train.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)','Date'])
train_y = m_train['price(원/kg)']
test_x  = m_test.drop(columns=['ID', 'timestamp','Date'])

In [11]:
qual_col = ['item', 'corporation', 'location','day_name']

test_x  = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)
test_x  = test_x.drop(columns=['기간','품목명'])
# for i in qual_col:
#     le = LabelEncoder()
#     train_x[i]=le.fit_transform(train_x[i])
#     test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

In [12]:
test_x  = test_x.drop(columns=['기간','품목명'])

In [16]:
low_corr = ['week', 'day', 'year']

In [17]:
# train_x = train_x.drop(columns=low_corr)
# test_x = test_x.drop(columns=low_corr)

In [18]:
reg  = create_model('rf')

NameError: name 'create_model' is not defined

In [20]:
tuned_clf = tune_model(reg, optimize = 'R2')
predictions = predict_model(tuned_clf, data = test_x)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,371.8575,817845.6805,904.3482,0.7986,2.5642,0.2359
1,405.9128,952131.1378,975.7721,0.7807,2.6478,0.2394
2,382.1366,852691.2792,923.4128,0.7704,2.6533,0.2481
3,389.1678,853284.8515,923.7342,0.7821,2.6919,0.2429
4,371.5445,797376.4065,892.9594,0.8150,2.5812,0.2329
5,406.9775,912719.3246,955.3635,0.7824,2.6403,0.2365
6,387.7355,884768.8901,940.6215,0.7824,2.6476,0.2360
7,383.5733,877848.1871,936.9355,0.7934,2.6351,0.2328
8,385.5864,858184.6686,926.3826,0.7939,2.5978,0.2471


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [25]:
result = pd.concat([test_re, predictions['prediction_label']], axis = 1)

result['answer'] = result['prediction_label'].copy()

del result['timestamp']
del result['item']
del result['corporation']
del result['location']
del result['year']
del result['month']
del result['day']
del result['Date']
del result['week']
del result['day_name']
del result['prediction_label']

In [26]:
926.9668

926.9668

In [27]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

result.head(20)

,ID,answer
0,TG_A_J_20230304,3579.63
1,TG_A_J_20230305,0.00
2,TG_A_J_20230306,3450.50
3,TG_A_J_20230307,3170.27
4,TG_A_J_20230308,3437.59
5,TG_A_J_20230309,3510.97
6,TG_A_J_20230310,3332.91
7,TG_A_J_20230311,3173.61
8,TG_A_J_20230312,0.00
9,TG_A_J_20230313,3305.44


In [28]:
result.to_csv('result2_tuned.csv', index = False)